# Lab02: Web Crawler (Continue).

- MSSV: 18600187
- Họ và tên: Vũ Cao Nguyên

## Yêu cầu bài tập

**Cách làm bài**


Bạn sẽ làm trực tiếp trên file notebook này; trong file, từ `TODO` để cho biết những phần mà bạn cần phải làm.

Bạn có thể thảo luận ý tưởng cũng như tham khảo các tài liệu, nhưng *code và bài làm phải là của bạn*. 

Nếu vi phạm thì sẽ bị 0 điểm cho bài tập này.

**Cách nộp bài**

Trước khi nộp bài, rerun lại notebook (`Kernel` -> `Restart & Run All`).

Sau đó, tạo thư mục có tên `MSSV` của bạn (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) Chép file notebook, file `output.txt` và file data các bạn đã lưu vào rồi nén thư mục `MSSV` này lại và nộp trên moodle.

**Nội dung bài tập**

Thu thập và thể hiện dữ liệu web.

## 2. Cài đặt

### 2.1. Import library

In [15]:
import requests
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import string

### 2.2. HTML Parser


Bộ phân tích cú pháp HTML (HTML Parser): nhận HTML code và trích xuất thông tin liên quan như tiêu đề của trang, đoạn văn trong trang, tiêu đề trong trang, liên kết, văn bản in đậm, v.v.

In [16]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
    def handle_starttag(self, tag, attrs):
        print("Start tag:", tag)

    def handle_endtag(self, tag):
        print("End tag :", tag)

    def handle_data(self, data):
        print("Data  :", data)

parser = MyHTMLParser()
parser.feed('<html><head><title>Test</title></head>')

Start tag: html
Start tag: head
Start tag: title
Data  : Test
End tag : title
End tag : head


#### Loại bỏ các HTML tag không cần thiết bằng cách thiết lập filter

In [17]:
def text_filter(element):
    if element.parent.name in ['style', 'title', 'script', 'head', '[document]', 'class', 'a', 'li']:
        return False
    elif isinstance(element, Comment):
        '''Opinion mining?'''
        return False
    elif re.match(r"[\s\r\n]+",str(element)): 
        '''space, return, endline'''
        return False
    return True

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 2.4 Thu thập nội dung trang Web

#### Trong bài tập này ta thể hiện tài liệu (hay nội dung trang web) với cấu trúc đơn giản như sau: 
- Gọi $D$ là một tập tài liệu chứa *n* tài liệu: $D=\left\{d_1,d_2,...,d_n\right\}$.
- Ta thể hiện tài liệu bằng một dictionary `data={}` với `data[word]=[[url_idx_1,url_idx_2,...],frequency]` với `url_index`$\in{\left[{1,n}\right]}$

VD: `data['at']=[[1,2], 5]`
Từ `at` xuất hiện trong đường dẫn có index `1` và `2` tổng số lần xuất hiện là 5.

#### Bước 1: liệt kê các từ xuất hiện trong trang web:

In [29]:
def wordList(url):
    r=requests.get(url)
    soup=BeautifulSoup(r.content,"html.parser")
    text=soup.findAll(text=True)
    filtered_text=list(filter(text_filter,text))
    word_list=[]

    #TODO:
    #c in string.punctuation
    for ch in filtered_text:
      for index in ch:
        if index in string.punctuation:
          ch = ch.replace(index,' ')
      ch = ch.split()
      for index in ch:
        word_list.append(index)

    return word_list

#### Bước 2: Tính tần suất xuất hiện của từ trong 1 trang web, lưu trữ dữ liệu vào data:

In [32]:
def read_url(url, url_idx, data):
    word_list = wordList(url)

    for index in word_list:
      index = index.lower()
      if data.get(index) is None:
        data[index] = [[url_idx],1]
      else:
        if url_idx in data[index][0]:
          data[index][1]+=1
        else:
          data[index][0].append(url_idx)
    
    #TODO
    '''Initialize value: data[w] = [[url_idx], 1]'''
                                    #list_url
    #VD: data.get("at")==None => initialize data["at"]
        # data.get("at")!=None >= add url_idx to list_url_idx (remember to check if this url_idx is in list_url_idx), 
                                # frequence+=1
    return 1  

#### Bước 3: Chạy chương trình lưu kết quả vào file output.txt

In [33]:
data={}
read_url('https://en.wikipedia.org/wiki/Web_mining', 1,data)
read_url('https://en.wikipedia.org/wiki/Data_mining', 2,data)

sorted_keys = sorted(data.keys())

with open("output.txt", "w",encoding="utf-8") as f:
    output_line = "Word".ljust(20) + "Frequency".ljust(15) + "URL_idx".ljust(15) + "\n"
    f.writelines(output_line)
    f.writelines('---------------------------------------------------------------------\n\n')
    for key in sorted_keys:
        output_string = str(key).ljust(20) + str(data[key][1]).ljust(15) + str(data[key][0]).ljust(15) + "\n"
        f.writelines(output_string)

In [38]:
#TODO
#Use python pickle save data. Load and print 10 first data values
# with open(dumpfile_path, "rb") as f:
import pickle

with open('18600187', "wb") as f:
  pickle.dump(data, f)
with open('18600187', "rb") as f:
  data = pickle.load(f)